#**Zajęcia nr 8**
## Projekt Machine Learning *Titanic - Machine Learning from Disaster*

Zajęcia nr 8 podsumowywały poruszony temat Machine Learning. Należało wziąć udział w dowolnym konkursie z działu *Compete* na stronie Kaggle. Polegało to na stworzeniu własnego modelu i zgłoszeniu go do konkursu. Następnie po zgłoszeniu model był oceniany i otrzymywał punktację - im bliżej wartości 1, tym model dokładniejszy, a więc lepszy. 


##**Zadanie do wykonania**

---
Postanowiono wziąć udział w konkursie o nazwie *Titanic - Machine Learning from Disaster* ze względu na jego przystępność oraz dobre wprowadzenie w zagadnienie *Competition*. Jako, że istnieje możliwość zgłaszania wielu modeli do jednego konkursu, stworzono ich kilka i zebrano oceny, jakie otrzymały po zgłoszeniu.  



###**Wprowadzenie**

W konkurencji należało przewidzieć na podstawie danych pasażerów biorących udział w rejsie statku Titanic, kto z nich przeżył katastrofę. 

Zapoznano się z danymi udostępnionymi w zakładce konkursu. zebrane zostały one w 3 plikach .csv:
- *train* - zawierał dane, które można było użyć do uczenia modelu takie jak imię, nazwisko, wiek czy ID pokładowe pasażerów oraz informację o tym, **czy przeżyli**
- *test* - to dane ponad 400 pasażerów użyte do przewidywania przez model, kto z nich przeżył. **Nie zawierały one informacji o tym, kto przeżył**
- *gender_submission* - plik pokazowy służący do sprawdzenia, jak powinny wyglądać prawidłowo przygotowane dane konkursowe. Stanowił on pierwsze zgłoszenie konkursowe. 







###**Pierwsze zgłoszenie konkursowe - model płciowy**

W celu przećwiczenia procedury wysyłania zgłoszenia, użyto pliku *gender_submission*, zawierającego dwie kolumny: ID pasażera *PassengerID* oraz oznaczenie, czy przeżył -  *Survived*. Dzielił on pasażerów pod względem płci następująco:
- kobieta - pasażer przeżył, 1 w kolumne *Survived*
- mężczyzna - pasażer nie przeżył, 0 w kolumne *Survived* 

Aby sprawdzić, czy przewidywania są słuszne, stworzono Notebook Kaggle, który domyślnie skonfigurowany jest następująco:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

wczytano do niego dane *train* oraz *test* oraz sprawdzono ich poprawność:

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

Obliczono procent pasażerów poszczególnych płci, którzy przeżyli:

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

Tabela z wynikami:

| Płeć      | Przeżywalność w % |
|-----------|-------------------|
| Kobieta   | 74.2              |
| Mężczyzna | 18.8              |

Jak widać, płeć wydawała się być bardzo ważnym czynnikiem pozwalającym na przeżycie katastrofy przez pasażera, zatem dobrym pomysłem jest użycie jej jako jednego z parametrów kolejnych modeli. Po zgłoszeniu pliku z przewidywaniami, otrzymał on punktację 0.76555. 

###**Model 1 - *Decision Tree Classifier*** 

Zaczęto od pierwszego typu modelu uczenia maszynowego o nazwie *Decision Tree Regressor*. Ma on postać drzewa decyzyjnego z gałęziami pnącymi się w dół. Każde rozgałęzienie to podział danych wejściowych na dwie części w zależności od parametru, który ma decydować o podziale i ich klasyfikacja. 

Poniżej (oraz w kolejnych podpunktach) przedstawiono część programu odpowiedzialną za naukę modelu - reszta kodu wyglądała podobnie jak w punkcie **Pierwsze zgłoszenie konkursowe**

**Parametry modelu**
- `max_depth=5` - maksymalna ilość rozgałęzień drzewa
- `random_state=1` - zapobiega losowości w trenowaniu modelu, przez co uzyskiwany wynik za każdym razem jest taki sam

In [ ]:
from sklearn.tree import DecisionTreeClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = DecisionTreeClassifier(random_state=1, max_depth = 5)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('DTRsubmission.csv', index=False)
print("Your submission was successfully saved!")

Otrzymany wynik za zgłoszenie przewidywań tego modelu to 0.77033

###**Model 2 - *Random Forest Classifier*** 

Kolejny typ modelu to *Random Forest Classifier*, czyli tworzenie wielu drzew taich jak w modelu 1 z losowymi parametrami decydującymi o rozgałęzieniach, a następnie klasyfikacja przez każde drzewo przeżywalności pasażera. Na koniec w wyniku demokratycznego wyboru, sumując predycykcje każdego z drzew, ustalane jest przewidywanie końcowe.

####**Wersja 1**

**Parametry modelu**
- `n_estimators=100` - ilość wygenerowanych drzew
- `max_depth=5` - maksymalna ilość rozgałęzień drzewa
- `random_state=1` - zapobiega losowości w trenowaniu modelu, przez co uzyskiwany wynik za każdym razem jest taki sam

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Otrzymany wynik: 0.77511

####**Wersja 2**

**Parametry modelu**
- `n_estimators=500`
- `max_depth=7`
- `random_state=1`

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=500, max_depth=7, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)


output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Otrzymany wynik: 0.77751

###**Model 3 - *Extra Trees Classifier*** 

Ostatni model jaki wzięto pod uwagę, to *Extra Trees Classifier*. Tworzy on szereg drzew decyzyjnych, które używają do przewidywań podzbioru zestawu danych do uczenia. Następnie wykorzystane zostaje uśrednianie, które zapobiega zjawisku *over-fittingu*.

**Parametry modelu**

**Parametry modelu**
- `n_estimators=100` - ilość wygenerowanych drzew
- `max_depth=5` - maksymalna ilość rozgałęzień drzewa
- `random_state=1` - zapobiega losowości w trenowaniu modelu, przez co uzyskiwany wynik za każdym razem jest taki sam

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = ExtraTreesClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('ETCsubmission.csv', index=False)
print("Your submission was successfully saved!")

Wynik po zgłoszeniu predykcji: 0.77272

###**Podsumowanie**

Poniżej przedstawiono tabelę z zebranymi wynikami każdego z modeli:

| Model                       | Wynik   |
|-----------------------------|---------|
| Model płciowy               | 0.76555 |
| Decision Tree Classifier    | 0.77033 |
| Random Forest Classifier v1 | 0.77511 |
| Random Forest Classifier v2 | 0.77751 |
| Extra Trees Classifier      | 0.77272 |

Wnioski:
- najlepszy wynik otrzymał model używający *Random Forest Classifier* w wersji 2. Miało to zapewne związek z większą ilością generowanych drzew oraz ich wielkością. Przy zmianie parametru `max_depth` należało mieć na uwadze zjawisko *over - fittingu*: model mógł być zbyt dokładny, przez co tracił zdolność uogólniania
- dla takich samych wartości parametrów Random Forest Classifier oraz Extra Trees Classifier lepiej spisał się ten pierwszy.
- przewidywanie na podstawie płci otrzymało wysoki wynik w porównaniu z innymi modelami, pozostając przy tym najmniej skommplikowaną metodą.
- wszystkie przetestowane modele (oprócz modelu płciowego) otrzymały wyniki nieznacznie różniące się od siebie - ocena różniła się dopiero od 3 miejsca po przecinku. 
